In [42]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
from dask.delayed import delayed
from zipfile import ZipFile
import hvplot.pandas
import hvplot.dask
import random
from datetime import date, datetime, timedelta
from tqdm import tqdm
import gzip
from scipy.sparse import csr_matrix, hstack, save_npz, load_npz

In [2]:
def load_zipped_csv(file_path):
    zipped_file = [delayed(pd.read_csv)(file_path)]
    return dd.from_delayed(zipped_file)

In [3]:
def add_total_number_of_days(df):
    df_to_join = df.groupby("user_id").days_since_prior_order.sum().reset_index().rename(columns={"days_since_prior_order":"total_days_on_platform"})
    df_to_join['user_id'] = df_to_join['user_id'].astype(int)
    df = pd.merge(df, df_to_join, on=["user_id"])
    return df

In [4]:
def add_initial_date(df):
    pass

In [5]:
def wrap_create_sparse_df(df, n_cols, n_rows, col_to_sparse_df, value_to_sparse_df, row_id):
    df_sparse = pd.DataFrame()
    for i in tqdm(range(n_cols)):
    #     print("product_id", i+1)
        a = np.empty((n_rows))
        a[:] = 0

        df_product = df[df[col_to_sparse_df] == i+1]
        for c in df_product[value_to_sparse_df].unique():
            df_p = df_product[df_product["count"] == c]
    #         print(f"Found {len(df_p['user_id'].to_list())} users who bought {c} of this product")
            a[df_p[row_id].to_list()] = c
        df_sparse[str(i)] = pd.arrays.SparseArray(a, dtype=float, fill_value=0)
    return df_sparse

In [6]:
def create_sparse_df(df, col_to_sparse_df, row_id):
    value_to_sparse_df = "count"
    df_exploded = df.explode(col_to_sparse_df)
    df_exploded[value_to_sparse_df] = 1
    
    df_exploded_grouped = df_exploded.groupby([col_to_sparse_df, row_id])[value_to_sparse_df].sum().reset_index()
    n_cols = len(df_exploded_grouped[col_to_sparse_df].unique())
    n_rows = len(df_exploded_grouped[row_id].unique()) + 1
    
    df_sparse = wrap_create_sparse_df(df_exploded_grouped, n_cols, n_rows, col_to_sparse_df, value_to_sparse_df, row_id)
    return df_sparse

# Load the data

In [7]:
df_order_products = load_zipped_csv("../data/order_products__prior.csv.zip")
df_orders = load_zipped_csv("../data/orders.csv.zip")
df_products = load_zipped_csv("../data/products.csv.zip")
df_aisles = load_zipped_csv("../data/aisles.csv.zip")
df_departments = load_zipped_csv("../data/departments.csv.zip")

In [8]:
df_orders = df_orders[df_orders["eval_set"] == "prior"]
df_orders['order_id'] = df_orders['order_id'].astype(int)

df_order_products = df_order_products.groupby("order_id")["product_id"].apply(list).compute().to_frame().reset_index()
df_order_products["len_product_list"] = df_order_products["product_id"].apply(len)
df_order_products['order_id'] = df_order_products['order_id'].astype(int)

/tmp/ipykernel_57919/1242674071.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  df_order_products = df_order_products.groupby("order_id")["product_id"].apply(list).compute().to_frame().reset_index()


In [9]:
df = df_orders.merge(df_order_products, on=["order_id"]).compute()

In [10]:
df = add_total_number_of_days(df)

# Create segmentation dataset

In [11]:
df.head(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,len_product_list,total_days_on_platform
0,2539329,1,prior,1,2,8,NaN,"[196, 14084, 12427, 26088, 26405]",5,176.0
1,2398795,1,prior,2,3,7,15.0,"[196, 10258, 12427, 13176, 26088, 13032]",6,176.0
2,473747,1,prior,3,3,12,21.0,"[196, 12427, 10258, 25133, 30450]",5,176.0
3,2254736,1,prior,4,4,7,29.0,"[196, 12427, 10258, 25133, 26405]",5,176.0
4,431534,1,prior,5,4,15,28.0,"[196, 12427, 10258, 25133, 10326, 17122, 41787...",8,176.0


In [12]:
nb_users = len(df["user_id"].unique())
nb_users

206209

## User dataset

In [13]:
df_users = df.groupby("user_id").agg(order_number_max=("order_number","max"), 
                                     order_dow_med=("order_dow","median"),
                                     order_dow_std=("order_dow","std"),
                                     order_dow_min=("order_dow","min"),
                                     order_dow_max=("order_dow","max"),
                                     days_since_prior_order_med=("days_since_prior_order","median"),
                                     total_days_on_platform_max=("total_days_on_platform","max"),
                                     len_product_list_med=("len_product_list","median"),
                                     len_product_list_std=("len_product_list","std"),
                                     len_product_list_min=("len_product_list","min"),
                                     len_product_list_max=("len_product_list","max"))
df_users = df_users.reset_index(drop=True)
df_users

,order_number_max,order_dow_med,order_dow_std,order_dow_min,order_dow_max,days_since_prior_order_med,total_days_on_platform_max,len_product_list_med,len_product_list_std,len_product_list_min,len_product_list_max
0,10,2.5,1.269296,1,4,20.0,176.0,5.5,1.523884,4,9
1,14,2.0,1.231456,1,5,13.0,198.0,13.5,5.717238,5,26
2,12,0.5,1.311372,0,3,11.0,133.0,7.0,2.103388,5,11
3,5,5.0,0.836660,4,6,17.0,55.0,3.0,2.073644,2,7
4,4,2.0,1.500000,0,3,11.0,40.0,10.0,3.095696,5,12
...,...,...,...,...,...,...,...,...,...,...,...
206204,3,4.0,1.527525,2,5,20.0,40.0,8.0,5.507571,7,17
206205,67,3.0,2.054988,0,6,3.0,249.0,4.0,3.230270,1,24
206206,16,2.0,2.096624,0,6,8.0,215.0,13.0,6.038419,5,25
206207,49,2.0,1.832019,0,6,7.0,357.0,13.0,5.592679,4,28


## Complete product dataset

In [14]:
df_products = df_products.merge(df_aisles.compute(), on=["aisle_id"])
df_products = df_products.merge(df_departments.compute(), on=["department_id"])
df_products = df_products.compute()
df_products

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,78,Nutter Butter Cookie Bites Go-Pak,61,19,cookies cakes,snacks
2,102,Danish Butter Cookies,61,19,cookies cakes,snacks
3,172,Gluten Free All Natural Chocolate Chip Cookies,61,19,cookies cakes,snacks
4,285,Mini Nilla Wafers Munch Pack,61,19,cookies cakes,snacks
...,...,...,...,...,...,...
49683,22827,Organic Black Mission Figs,18,10,bulk dried fruits vegetables,bulk
49684,28655,Crystallized Ginger Chunks,18,10,bulk dried fruits vegetables,bulk
49685,30365,Vegetable Chips,18,10,bulk dried fruits vegetables,bulk
49686,38007,Naturally Sweet Plantain Chips,18,10,bulk dried fruits vegetables,bulk


In [15]:
nb_products = len(df_products["product_id"].unique())
nb_products

49688

## Sparse user dataset - products

In [16]:
df_product_to_sparse = df[["user_id", "product_id"]]
df_sparse_product = create_sparse_df(df_product_to_sparse, col_to_sparse_df="product_id", row_id="user_id")
del df_product_to_sparse
df_sparse_product

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49677/49677 [17:19<00:00, 47.79it/s]


,0,1,2,3,4,5,6,7,8,9,...,49667,49668,49669,49670,49671,49672,49673,49674,49675,49676
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
206206,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
206207,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
206208,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
df_aisles_to_sparse = df[["user_id", "product_id"]]
df_aisles_to_sparse = df_aisles_to_sparse.explode("product_id")
df_aisles_to_sparse = df_aisles_to_sparse.merge(df_products[["product_id", "aisle_id"]], on=["product_id"])
df_sparse_aisle = create_sparse_df(df_aisles_to_sparse, col_to_sparse_df="aisle_id", row_id="user_id")
del df_aisles_to_sparse
df_sparse_aisle

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 134/134 [00:04<00:00, 28.53it/s]


,0,1,2,3,4,5,6,7,8,9,...,124,125,126,127,128,129,130,131,132,133
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
206206,0.0,3.0,7.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
206207,0.0,0.0,6.0,0.0,0.0,0.0,2.0,0.0,6.0,0.0,...,1.0,0.0,0.0,2.0,0.0,3.0,1.0,0.0,0.0,0.0
206208,0.0,0.0,35.0,11.0,2.0,0.0,1.0,0.0,2.0,0.0,...,0.0,0.0,0.0,7.0,0.0,2.0,2.0,0.0,0.0,0.0


In [18]:
df_department_to_sparse = df[["user_id", "product_id"]]
df_department_to_sparse = df_department_to_sparse.explode("product_id")
df_department_to_sparse = df_department_to_sparse.merge(df_products[["product_id", "department_id"]], on=["product_id"])
df_sparse_departments = create_sparse_df(df_department_to_sparse, col_to_sparse_df="department_id", row_id="user_id")
del df_department_to_sparse
df_sparse_departments

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:01<00:00, 10.77it/s]


,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,5.0,0.0,0.0,13.0,0.0,0.0,0.0,...,0.0,1.0,3.0,0.0,13.0,2.0,0.0,22.0,0.0,0.0
2,17.0,0.0,2.0,36.0,0.0,3.0,9.0,0.0,0.0,0.0,...,1.0,11.0,3.0,1.0,48.0,0.0,0.0,42.0,21.0,0.0
3,6.0,0.0,0.0,38.0,0.0,0.0,3.0,0.0,4.0,0.0,...,0.0,4.0,0.0,0.0,21.0,1.0,0.0,9.0,2.0,0.0
4,3.0,0.0,2.0,2.0,2.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206205,2.0,0.0,2.0,7.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,14.0,0.0,1.0,0.0,3.0,1.0
206206,76.0,1.0,2.0,34.0,0.0,4.0,33.0,0.0,1.0,0.0,...,2.0,14.0,1.0,9.0,39.0,12.0,0.0,42.0,5.0,0.0
206207,21.0,0.0,3.0,50.0,0.0,1.0,20.0,0.0,8.0,0.0,...,6.0,15.0,4.0,11.0,52.0,0.0,1.0,23.0,8.0,0.0
206208,26.0,0.0,62.0,197.0,0.0,3.0,20.0,0.0,17.0,0.0,...,22.0,38.0,17.0,11.0,161.0,6.0,3.0,59.0,28.0,2.0


## Combine dataframes

In [19]:
df_sparse_users = csr_matrix(df_users.astype(pd.SparseDtype(float, fill_value=0)).sparse.to_coo())
df_sparse_product = csr_matrix(df_sparse_product.sparse.to_coo())
df_sparse_aisle = csr_matrix(df_sparse_aisle.sparse.to_coo())
df_sparse_departments = csr_matrix(df_sparse_departments.sparse.to_coo())

In [29]:
print(df_sparse_users.shape)
print(df_sparse_product.shape)
print(df_sparse_aisle.shape)
print(df_sparse_departments.shape)

(206209, 11)
(206210, 49677)
(206210, 134)
(206210, 21)


In [39]:
segmentation_data = hstack([df_sparse_users, df_sparse_product[1:, :], df_sparse_aisle[1:, :] ,df_sparse_departments[1:, :]])

In [40]:
segmentation_data.shape

(206209, 49843)

In [43]:
save_npz('sparse_matrix.npz', segmentation_data)

In [44]:
segmentation_data = load_npz('sparse_matrix.npz')
segmentation_data.shape

(206209, 49843)

In [20]:
# f = gzip.GzipFile("df_sparse_departments.npy.gz", "w")
# np.save(file=f, arr=np.array(df_sparse_departments))
# f.close()

In [21]:
# f = gzip.GzipFile('df_sparse_departments.npy.gz', "r")
# a = np.load(f)